# Module 1 - Programming Assignment

## General Directions

1. You must follow the Programming Requirements outlined on Canvas.
2. The Notebook should be cleanly and fully executed before submission.
3. You should change the name of this file to be your JHED id. For example, `jsmith299.ipynb` although Canvas will change it to something else... :/

<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        You should always read the entire assignment before beginning your work, so that you know in advance what the requested output will be and can plan your implementation accordingly.
    </p>
</div>

# State Space Search with A* Search

You are going to implement the A\* Search algorithm for navigation problems.


## Motivation

Search is often used for path-finding in video games. Although the characters in a video game often move in continuous spaces,
it is trivial to layout a "waypoint" system as a kind of navigation grid over the continuous space. Then if the character needs
to get from Point A to Point B, it does a line of sight (LOS) scan to find the nearest waypoint (let's call it Waypoint A) and
finds the nearest, LOS waypoint to Point B (let's call it Waypoint B). The agent then does a A* search for Waypoint B from Waypoint A to find the shortest path. The entire path is thus Point A to Waypoint A to Waypoint B to Point B.

We're going to simplify the problem by working in a grid world. The symbols that form the grid have a special meaning as they
specify the type of the terrain and the cost to enter a grid cell with that type of terrain:

```
token   terrain    cost 
🌾       plains     1
🌲       forest     3
🪨       hills      5
🐊       swamp      7
🗻       mountains  impassible
```

We can think of the raw format of the map as being something like:

```
🌾🌾🌾🌾🌲🌾🌾
🌾🌾🌾🌲🌲🌲🌾
🌾🗻🗻🗻🌾🌾🌾
🌾🌾🗻🗻🌾🌾🌾
🌾🌾🗻🌾🌾🌲🌲
🌾🌾🌾🌾🌲🌲🌲
🌾🌾🌾🌾🌾🌾🌾
```

## The World

Given a map like the one above, we can easily represent each row as a `List` and the entire map as `List of Lists`:

In [1]:
full_world = [
['🌾', '🌾', '🌾', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🗻', '🗻', '🗻', '🗻', '🗻', '🗻', '🗻', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🗻', '🗻', '🗻', '🪨', '🪨', '🪨', '🗻', '🗻', '🪨', '🪨'],
['🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🐊', '🐊', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🪨', '🌾'],
['🌾', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🌲', '🌲', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🪨', '🌾'],
['🌾', '🪨', '🪨', '🪨', '🗻', '🗻', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🪨', '🌾', '🌾'],
['🌾', '🪨', '🪨', '🗻', '🗻', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🪨', '🪨', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🌾', '🐊', '🐊', '🌾', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🪨', '🪨', '🗻', '🗻', '🗻', '🗻', '🌾', '🌾', '🌾', '🐊', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🪨', '🪨', '🗻', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🗻', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🌾', '🌾', '🪨', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾'],
['🐊', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🪨', '🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🌾', '🪨', '🗻', '🪨', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🌲', '🌲', '🪨', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🗻', '🌾', '🌾', '🌲', '🌲', '🌲', '🌲', '🪨', '🪨', '🪨', '🪨', '🌾', '🐊', '🐊', '🐊', '🌾', '🌾', '🪨', '🗻', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🗻', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🗻', '🗻', '🗻', '🪨', '🪨', '🌾', '🐊', '🌾', '🪨', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🗻', '🗻', '🗻', '🌾', '🌾', '🗻', '🗻', '🗻', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🗻', '🗻', '🗻', '🗻', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🗻', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
['🌾', '🌾', '🌾', '🌾', '🗻', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '🪨', '🪨', '🪨', '🪨', '🗻', '🗻', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾', '🗻', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🗻', '🗻', '🗻', '🌲', '🌲', '🗻', '🗻', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🗻', '🗻', '🗻', '🗻', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🌾', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🌾', '🪨', '🪨', '🌾', '🌾', '🪨', '🪨', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🪨', '🪨', '🌾', '🐊', '🐊', '🐊', '🐊', '🐊'],
['🪨', '🗻', '🪨', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🗻', '🗻', '🗻', '🪨', '🪨', '🗻', '🗻', '🌾', '🗻', '🗻', '🪨', '🪨', '🐊', '🐊', '🐊', '🐊'],
['🪨', '🗻', '🗻', '🗻', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🗻', '🗻', '🗻', '🗻', '🪨', '🪨', '🪨', '🪨', '🗻', '🗻', '🗻', '🐊', '🐊', '🐊', '🐊'],
['🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾', '🌾', '🪨', '🪨', '🪨', '🌾', '🌾', '🌾']
]

## Warning

One implication of this representation is that (x, y) is world[ y][ x] so that (3, 2) is world[ 2][ 3] and world[ 7][ 9] is (9, 7). Yes, there are many ways to do this. I picked this representation because when you look at it, it *looks* like a regular x, y cartesian grid and it's easy to print out.

It is often easier to begin your programming by operating on test input that has an obvious solution. If we had a small 7x7 world with the following characteristics:

In [2]:
small_world = [
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌲', '🌲', '🌲', '🌲', '🌲', '🌲'],
    ['🌾', '🌾', '🌾', '🌾', '🌾', '🌾', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾'],
    ['🌲', '🌲', '🌲', '🌲', '🌲', '🌲', '🌾']
]

**what do you expect the policy would be?** Think about it for a bit. This will help you with your programming and debugging.

## States and State Representation

The canonical pieces of a State Space Search problem are the States, Actions, Transitions and Costs. 

We'll start with the state representation. For the navigation problem, a state is the current position of the agent, `(x,y)`. The entire set of possible states is implicitly represented by the world map.

## Actions and Transitions

Next we need to specify the actions. In general, there are a number of different possible action sets in such a world. The agent might be constrained to move north/south/east/west or diagonal moves might be permitted as well (or really anything). When combined with the set of States, the *permissible* actions forms the Transition set.

Rather than enumerate the Transition set directly, for this problem it's easier to calculate the available actions and transitions on the fly. This can be done by specifying a *movement model* as offsets to the current state and then checking to see which of the potential successor states are actually permitted. This can be done in the successor function mentioned in the pseudocode.

One such example of a movement model is shown below.

In [3]:
MOVES = [(0,-1), (1,0), (0,1), (-1,0)]

## Costs

We can encode the costs described above in a `Dict`:

In [4]:
COSTS = { '🌾': 1, '🌲': 3, '🪨': 5, '🐊': 7,' ':0}

## Specification

You will implement a function called `a_star_search` that takes the parameters and returns the value as specified below. The return value is going to look like this:

`[(0,1), (0,1), (0,1), (1,0), (1,0), (1,0), (1,0), (1,0), (1,0), (0,1), (0,1), (0,1)]`

You should also implement a function called `pretty_print_path`. 
The `pretty_print_path` function prints an ASCII representation of the path generated by the `a_star_search` on top of the terrain map. 
For example, for the test world, it would print this:

```
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏩⏩⏩⏩⏩⏩⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲🎁
```

using ⏩,⏪,⏫ ⏬ to represent actions and `🎁` to represent the goal. (Note the format of the output...there are no spaces, commas, or extraneous characters). You are printing the path over the terrain.
This is an impure function (because it has side effects, the printing, before returning anything).

Note that in Python:
```
> a = ["*", "-", "*"]
> "".join(a)
*-*
```
Do not print raw data structures; do not insert unneeded/requested spaces!

### Additional comments

As Python is an interpreted language, you're going to need to insert all of your functions *before* the actual `a_star_search` function implementation. 
Do not make unwarranted assumptions (for example, do not assume that the start is always (0, 0).
Do not refer to global variables, pass them as parameters (functional programming).

Simple and correct is better than inefficient and incorrect, or worse, incomplete.
For example, you can use a simple List, with some helper functions, as a Stack or a Queue or a Priority Queue.
Avoid the Python implementations of HeapQ, PriorityQueue implementation unless you are very sure about what you're doing as they require *immutable* keys.

In [5]:
from typing import List, Tuple, Dict, Callable
from copy import deepcopy
from collections import OrderedDict

*add as many markdown and code cells here as you need for helper functions. We have added `heuristic` for you*

<a id="heuristic"></a>
## heuristic
We know that in order for the heuristic to be valid, it must be both admissible and consistent. This indicates that the heuristic should never overestimate the cost to reach a goal (admissible) and the heuristic for a current state should be less than or equal to the cost of moving to the next state plus the next states' heuristic.
For the estimated cost we will use the manhattan distance which we implement taking into account the different between the x and y coordinates with respect to the goal. 
The function itself receives as parameters the world, location to be evaluated, the goal state, and the list of costs. It returns the integer value for the heuristic. 

In [6]:
def heuristic(world:list, location:tuple, goal:tuple, costs:list): # you can add formal parameters
    
    # get cost from dictionary
    cost = costs[world[location[0]][location[1]]]
    
    # Implement manhatta distance
    x_distance = abs(goal[1] - location[1])
    y_distance = abs(goal[0] - location[0])
    manhattan_distance = x_distance + y_distance
    
    heuristic = manhattan_distance + cost
    
    return heuristic

In [7]:
# assertions/unit tests
assert heuristic(small_world, (3,3), (6,6), COSTS) == 7 #Manhattan= 6-3+6-3, cost = 1
assert heuristic(small_world, (0,0), (6,6), COSTS) == 13 #Manhattan = 6+6, cost= 1
assert heuristic(small_world, (1,1), (6,6), COSTS) == 13 #Manhattan= 5+5, cost = 3

<a id="a_star_search"></a>
## a_star_search

*add documentation/description of the algorithm here; connect it to the theory!*

* **world** List[List[str]]: the actual context for the navigation problem.
* **start** Tuple[int, int]: the starting location of the bot, `(x, y)`.
* **goal** Tuple[int, int]: the desired goal position for the bot, `(x, y)`.
* **costs** Dict[str, int]: is a `Dict` of costs for each type of terrain in **world**.
* **moves** List[Tuple[int, int]]: the legal movement model expressed in offsets in **world**.
* **heuristic** Callable: is a heuristic function, $h(n)$.


**returns** List[Tuple[int, int]]: the offsets needed to get from start state to the goal as a `List`.

This functions takes as parameters the world, start state, goal state, the dictionary of costs, the moves available, and the heuristic function as previously defined. There is an important caveat and distinction to be made with regards to the actual implementation. Provided that two subsequent states have the same heuristic value, the direction of the search algorithm will depend on the order of the moves provided. Because the move have a significant impact, a brief modification can be implemented from those provided. 


In [8]:
def a_star_search( world: list, start: tuple, goal: tuple, costs: dict, moves: list, heuristic) -> list:
    ### YOUR SOLUTION HERE ###
    
    # place initial state on frontier
    frontier = [start]
    # initialize explored list
    explored = []
    
       
    def add_succesors(world, current_state, moves):
        successors = list()
        for i in moves:
            provisional_state = (current_state[0] + i[1],current_state[1] + i[0])
            if provisional_state[0] < 0 or provisional_state[1] < 0:
                continue            
            try:
                world[provisional_state[0]][provisional_state[1]]
                if world[provisional_state[0]][provisional_state[1]]=='🗻':
                    continue   
                successors.append(provisional_state)
            except:
                continue
        return successors               
    
    # while frontier is not empty
    while len(frontier)!=0:
        # current state := next state on frontier
        current_state = frontier.pop()
        # return path(current_state) if is-terminal(current_state)
        if current_state == goal:
            explored.append(current_state)
            return explored
        # children := successors(current_state)
        children = add_succesors(world, current_state, moves)
        # for each child in children
        provisional_frontier = {}
        for child in children:
            if child in explored:
                continue
            else:
                provisional_frontier[heuristic(world, child, goal, costs)] = child
        
        list_heuristics = [i for i in provisional_frontier.keys()]
        list_heuristics.sort()
        
        for i in provisional_frontier:            
            frontier.append(provisional_frontier[list_heuristics[0]])
                
        # add add current-state to explored
        explored.append(current_state)
    
    ### YOUR SOLUTION HERE ###
    return explored # change to return the real answer

## pretty_print_path

*write your documentation here*

* **world** List[List[str]]: the world (terrain map) for the path to be printed upon.
* **path** List[Tuple[int, int]]: the path from start to goal, in offsets.
* **start** Tuple[int, int]: the starting location for the path.
* **goal** Tuple[int, int]: the goal location for the path.
* **costs** Dict[str, int]: the costs for each action.

**returns** int - The path cost.

In this function we traverse through the rows and generate strings which are printed to the command. In order to compute the aggregate cost we traverse through the path and aggregate the individual costs into a singular integer value. 


<div style="background: lemonchiffon; margin:20px; padding: 20px;">
    <strong>Important</strong>
    <p>
        Does your output of pretty_print_path really look like the specification? Go check again.
    </p>
</div>

In [9]:
def pretty_print_path( world: list, path: list, start: tuple, goal: tuple, costs: dict) -> int:
    
    def arrow_path(path):
        path_dict = {}
        # MOVES = [(0,-1), (1,0), (0,1), (-1,0)]
        # ⏩,⏪,⏫,⏬
        for i in range(len(path[1:])):
            if path[i+1] == (path[i][0], path[i][1]-1):
                path_dict[path[i]]='⏪'
            elif path[i+1] == (path[i][0]+1, path[i][1]):
                path_dict[path[i]]='⏬'
            elif path[i+1] == (path[i][0]-1, path[i][1]):
                path_dict[path[i]]='⏫'
            else: 
                path_dict[path[i]]='⏩'
        return path_dict

    path_dict = arrow_path(path)
    
    for i in range(len(world)):
        row = ''
        for j in range(len(world[i])):
            if (i,j) == goal:
                row += '🎁'
            elif (i,j) in path:
                row += path_dict[(i,j)]
            else:
                row += world[i][j]
        print(row)
        
    cost = 0
    for i,j in path:
        cost += costs[world[i][j]]
            
    return cost # replace with the real value!

## Problems

## Problem 1. 

Execute `a_star_search` and `pretty_print_path` for the `small_world`.

If you change any values while developing your code, make sure you change them back! (Better yet, don't do it. Copy them elsewhere and change the values, then delete those experiments).

In [10]:
small_start = (0, 0)
small_goal = (len(small_world[0]) - 1, len(small_world) - 1)
small_path = a_star_search(small_world, small_start, small_goal, COSTS, MOVES, heuristic)
small_path_cost = pretty_print_path(small_world, small_path, small_start, small_goal, COSTS)
print(f"total path cost: {small_path_cost}")
print(small_path)

⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏬🌲🌲🌲🌲🌲🌲
⏩⏩⏩⏩⏩⏩⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲⏬
🌲🌲🌲🌲🌲🌲🎁
total path cost: 13
[(0, 0), (1, 0), (2, 0), (3, 0), (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6), (4, 6), (5, 6), (6, 6)]


## Problem 2

Execute `a_star_search` and `print_path` for the `full_world`

In [11]:
full_start = (0, 0)
full_goal = (len(full_world[0]) - 1, len(full_world) - 1)
full_path = a_star_search(full_world, full_start, full_goal, COSTS, MOVES, heuristic)
full_path_cost = pretty_print_path(full_world, full_path, full_start, full_goal, COSTS)
print(f"total path cost: {full_path_cost}")
print(full_path)

⏬🌾🌾🌾🌾🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾
⏬🌾🌾🌾🌾🌾🌾🌲🌲🌲🌲🌲🌲🌲🌲🌲🌾🌾🗻🗻🗻🗻🗻🗻🗻🌾🌾
⏬🌾🌾🌾🗻🗻🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🗻🗻🗻🪨🪨🪨🗻🗻🪨🪨
⏬🌾🌾🌾🪨🗻🗻🗻🌲🌲🌲🌲🐊🐊🌲🌲🌲🌲🌲🌾🌾🪨🪨🗻🗻🪨🌾
⏬🌾🌾🪨🪨🗻🗻🌲🌲🌾🌾🐊🐊🐊🐊🌲🌲🌲🌾🌾🌾🪨🗻🗻🗻🪨🌾
⏬🪨🪨🪨🗻🗻🪨🪨🌾🌾🌾🌾🐊🐊🐊🐊🐊🌾🌾🌾🌾🌾🪨🗻🪨🌾🌾
⏬🪨🪨🗻🗻🪨🪨🌾🌾🌾🌾🪨🗻🗻🗻🐊🐊🐊🌾🌾🌾🌾🌾🪨🌾🌾🌾
⏬🌾🪨🪨🪨🪨🪨🌾🌾🌾🌾🌾🌾🪨🗻🗻🗻🐊🐊🐊🌾🌾🪨🪨🪨🌾🌾
⏬🌾🌾🪨🪨🪨🌾🌾🌾🌾🌾🌾🪨🪨🗻🗻🌾🐊🐊🌾🌾🪨🪨🪨🌾🌾🌾
⏬🌾🌾🐊🐊🐊🌾🌾🪨🪨🪨🗻🗻🗻🗻🌾🌾🌾🐊🌾🪨🪨🪨🌾🌾🌾🌾
⏬🌾🐊🐊🐊🐊🐊🌾🪨🪨🗻🗻🗻🪨🌾🌾🌾🌾🌾🪨🗻🗻🗻🪨🌾🌾🌾
⏬🐊🐊🐊🐊🐊🌾🌾🪨🗻🗻🪨🌾🌾🌾🌾🐊🐊🌾🌾🪨🗻🗻🪨🌾🌾🌾
⏬🐊🐊🐊🐊🌾🌾🪨🪨🗻🗻🪨🌾🐊🐊🐊🐊🌾🌾🌾🪨🗻🪨🌾🌾🌾🌾
⏬🐊🐊🐊🐊🌾🌾🪨🌲🌲🪨🌾🌾🌾🌾🐊🐊🐊🐊🌾🌾🪨🌾🌾🌾🌾🌾
⏬🌾🌾🌾🗻🌾🌾🌲🌲🌲🌲🪨🪨🪨🪨🌾🐊🐊🐊🌾🌾🪨🗻🪨🌾🌾🌾
⏬🌾🌾🗻🗻🗻🌲🌲🌲🌲🌲🌲🗻🗻🗻🪨🪨🌾🐊🌾🪨🗻🗻🪨🌾🌾🌾
⏬🌾🗻🗻🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🗻🗻🗻🌾🌾🗻🗻🗻🌾🌾🌾🌾🌾
⏬🌾🌾🗻🗻🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🌲🗻🗻🗻🗻🌾🌾🌾🌾🌾🌾🌾
⏬🌾🌾🗻🗻🗻🌲🌲🌲🌲🌲🌲🌲🌲🌾🌾🌾🪨🪨🌾🌾🌾🌾🌾🌾🌾🌾
⏬🌾🌾🌾🗻🗻🗻🌲🌲🌲🌲🌲🌲🌾🌾🌾🌾🌾🌾🌾🌾🌾🌾🐊🐊🐊🐊
⏬🌾🪨🪨🪨🪨🗻🗻🌲🌲🌲🌲🌲🌾🗻🌾🌾🌾🌾🌾🐊🐊🐊🐊🐊🐊🐊
⏬🌾🌾🌾🪨🪨🪨🗻🗻🗻🌲🌲🗻🗻🌾🌾🌾🌾🌾🌾🐊🐊🐊🐊🐊🐊🐊
⏬🌾🌾🌾🌾🌾🪨🪨🪨🗻🗻🗻🗻🌾🌾🌾🌾🪨🪨🌾🌾🐊🐊🐊🐊🐊🐊
⏬🪨🪨🌾🌾🪨🪨🪨🪨🪨🌾🌾🌾🌾🌾🪨🪨🗻🗻🪨🪨🌾🐊🐊🐊🐊🐊
⏬🗻🪨🪨🪨🪨🌾🌾🌾🌾🌾🗻🗻🗻🪨🪨🗻🗻🌾🗻🗻🪨🪨🐊🐊🐊🐊
⏬🗻🗻🗻🪨🌾🌾🌾🌾🌾🪨🪨🗻🗻🗻🗻🪨🪨🪨🪨🗻🗻🗻🐊🐊🐊🐊
⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩⏩🎁
total path cost: 107
[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 0), (9, 0), (10, 0), (11, 0), (12, 0), (13, 0), (14, 0), (15, 0), (16, 0), (17, 0), (18, 0), (19, 0), (20, 0), (21, 0), (22, 0), (23, 0), (24, 0), (25, 0)

## Comments

I think that I can have a better implementation of the priority queue. The limitation of not using OOP left me somewhat short handed when trying to implement it. I believe my problem is related to tie breaking. 

## To think about for future assignments...

This first assignment may not have been difficult for you if you've encountered A* search before in your Algorithms course. In preparation for future assignments that build on State Space Search, you can think about the following or even do an implementation if you like. You should **not** submit it as part of this assignment.

In several future assignments, we will have a need for a "plain ol'" Depth First Search algorithm.

1. Implement DFS Search to solve the problem presented in this programming assignment. Try to be as general as possible (don't hard code anything you can pass as a formal parameter).
2. Can you implement DFS Search as a higher order function and supply your own `is_goal`, `successors`, and `path` functions? How do you handle *state*?
3. Can you write a version of DFS that returns all the solutions?

In one future assignment a Breadth First Search algorithm will be very handy. Can you implement a search algorithm that changes whether it uses DFS or BFS by parameterization?

## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you follow the Programming Requirements on Canvas?

Do not submit any other files.